Import the stuff we will need

In [ ]:
import numpy as np
from lsst.daf.persistence import Butler
from lsst.eotest.sensor import MaskedCCD, makeAmplifierGeometry
from lsst.eo_utils.base.image_utils import get_ccd_from_id,\
    get_amp_list, get_data_as_read

Define a function to test that we got the orientation correct

In [ ]:
def test_flips(bulter, exp, ccd, vendor):
    amps_det = get_amp_list(butler, exp)
    amps_ccd = get_amp_list(None, ccd)
    nfail = 0
    for amp_det, amp_ccd in zip(amps_det, amps_ccd):
        ff_det = get_data_as_read(butler, exp, amp_det)
        ff_ccd = get_data_as_read(None, ccd, amp_ccd)

        diff_i = ff_det['imaging'] - ff_ccd['imaging']
        if (np.abs(diff_i) > 0).any():
            print("Failed %s imaging for amp %i" % (vendor, amp_det))
            nfail += 1

        diff_p = ff_det['parallel_overscan'] - ff_ccd['parallel_overscan'][:,:-1]
        if (np.abs(diff_p) > 0).any():
            print("Failed %s parallel overscan for amp %i" % (vendor, amp_det))
            nfail += 1

        diff_s = ff_det['serial_overscan'][0:2000] - ff_ccd['serial_overscan'][0:2000]
        if (np.abs(diff_s) > 0).any():
            print("Failed %s parallel overscan for amp %i" % (vendor, amp_det))
            nfail += 1

    return nfail              

In [ ]:
# Set the path to the Butler repository and import a Butler
repo_path = "/gpfs/slac/lsst/fs3/g/data/datasets/ts8"
butler = Butler(repo_path)

In [ ]:
# Get a dataId for the first bias frame for a FE55 run
incompleteDataId_itl = dict(run='6106D', imageType='BIAS', testType='FE55', detectorName='S00')
incompleteDataId_e2v = dict(run='6006D', imageType='BIAS', testType='FE55', detectorName='S00')
ss_itl = butler.subset('raw', '', incompleteDataId_itl)
ss_e2v = butler.subset('raw', '', incompleteDataId_e2v)
dataId_itl = ss_itl.cache[0]
dataId_e2v = ss_e2v.cache[0]

In [ ]:
dataId_itl

In [ ]:
# Specify the corresponding file
fname_itl = '/gpfs/slac/lsst/fs1/g/data/jobHarness/jh_archive-test/LCA-11021_RTM/LCA-11021_RTM-004-Dev/6106D/fe55_raft_acq/v0/41316/S00/ITL-3800C-381-Dev_fe55_bias_000_6106D_20180825170428.fits'
fname_e2v = '/gpfs/slac/lsst/fs1/g/data/jobHarness/jh_archive-test/LCA-11021_RTM/LCA-11021_RTM-010-Dev/6006D/fe55_raft_acq/v0/40509/S00/E2V-CCD250-266-Dev_fe55_bias_000_6006D_20180724054018.fits'

In [ ]:
# Get the data using the Butler, and using MaskedCCD, for both ITL and E2V
exp_e2v = get_ccd_from_id(butler, dataId_e2v, [])
ccd_e2v = get_ccd_from_id(None, fname_e2v, [])

exp_itl = get_ccd_from_id(butler, dataId_itl, [])
ccd_itl = get_ccd_from_id(None, fname_itl, [])

In [ ]:
nfail_e2v = test_flips(butler, exp_e2v, ccd_e2v, "e2v")
nfail_itl = test_flips(butler, exp_itl, ccd_itl, "itl")

In [ ]:
if nfail_e2v + nfail_itl == 0:
    print("Success!")
else:
    print("Failures: %i e2v, %i itl." % (nfail_e2v, nfail_itl) )